# Captura de video

In [11]:
import cv2
import numpy as np
from ultralytics import YOLO
import math

# Ejemplo 1: Captura de camara

In [2]:
# Selecciono la camara por defecto (0).
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se puede abrir la cámara")
    exit()
# Bucle principal
while True:
    # Capturo frame a frame
    ret, frame = cap.read()
    if not ret:
        print("No se puede recibir el frame (stream end?). Saliendo ...")
        break

    frame = cv2.flip(frame, 1)
    cv2.imshow('Espejo', frame)

    # Presiono 'q' para salir del bucle.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Tecle 'q' para salir.")
        break

cap.release()
cv2.destroyAllWindows()

Tecle 'q' para salir.


# Ejemplo 2: Captura con deteccion de bordes

In [ ]:
# Selecciono la camara por defecto (0).
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se puede abrir la cámara")
    exit()
# Bucle principal
while True:
    # Capturo frame a frame
    ret, frame = cap.read()
    if not ret:
        print("No se puede recibir el frame (stream end?). Saliendo ...")
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)

    # Abre camara original
    cv2.imshow('Espejo', frame)
    #Abre camara con detector de bordes
    cv2.imshow('Detector de bordes', edges)

    # Presiono 'q' para salir del bucle.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Tecle 'q' para salir.")
        break

cap.release()
cv2.destroyAllWindows()

Tecle 'q' para salir.


# Ejemplo 3: Deteccion de rostros

In [5]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Selecciono la camara por defecto (0).
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("No se puede abrir la cámara")
    exit()
# Bucle principal
while True:
    # Capturo frame a frame
    ret, frame = cap.read()
    if not ret:
        print("No se puede recibir el frame (stream end?). Saliendo ...")
        break

    frame = cv2.flip(frame, 1)
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    cv2.imshow('Detección de rostros', frame)


    # Presiono 'q' para salir del bucle.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        print("Tecle 'q' para salir.")
        break

cap.release()
cv2.destroyAllWindows()

Tecle 'q' para salir.


# Deteccion de Rostros por RNN

In [6]:
prototxt_path = "deploy.prototxt.txt"
model_path = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Inicializo la camara.
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Detector de rostros (CNN) iniciado. Presione 'q' para salir.")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Volteo el frame.
    frame_flipped = cv2.flip(frame, 1)
    (h, w) = frame_flipped.shape[:2]


    blob = cv2.dnn.blobFromImage(cv2.resize(frame_flipped, (300, 300)), 1.0,
                               (300, 300), (104.0, 177.0, 123.0))


    net.setInput(blob)
    detections = net.forward()

    # Itero sobre las detecciones que encuentre.
    for i in range(0, detections.shape[2]):
        # Extraigo la confianza (probabilidad) de la deteccion.
        confidence = detections[0, 0, i, 2]

        # Filtro las detecciones debiles, me quedo solo con las que superan el 50% de confianza.
        if confidence > 0.5:
            # Calculo las coordenadas del cuadro delimitador.
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Dibujo el cuadro y la confianza sobre el frame.
            text = f"{confidence * 100:.2f}%"
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame_flipped, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame_flipped, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)

    cv2.imshow("Detector de Rostros - CNN", frame_flipped)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


Detector de rostros (CNN) iniciado. Presione 'q' para salir.


# Grabacion de video

In [ ]:
prototxt_path = "deploy.prototxt.txt"
model_path = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(prototxt_path, model_path)

# Inicializo la camara.
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

print("Detector de rostros (CNN) iniciado. Presione 'q' para salir.")

# Configuro el video para grabar
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = cv2.VideoWriter_fourcc(*'XVID')
out = cv2.VideoWriter('grabacion_deteccion.avi', fourcc, 20.0,(frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Volteo el frame.
    frame_flipped = cv2.flip(frame, 1)
    (h, w) = frame_flipped.shape[:2]


    blob = cv2.dnn.blobFromImage(cv2.resize(frame_flipped, (300, 300)), 1.0,
                               (300, 300), (104.0, 177.0, 123.0))


    net.setInput(blob)
    detections = net.forward()

    # Itero sobre las detecciones que encuentre.
    for i in range(0, detections.shape[2]):
        # Extraigo la confianza (probabilidad) de la deteccion.
        confidence = detections[0, 0, i, 2]

        # Filtro las detecciones debiles, me quedo solo con las que superan el 50% de confianza.
        if confidence > 0.5:
            # Calculo las coordenadas del cuadro delimitador.
            box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
            (startX, startY, endX, endY) = box.astype("int")

            # Dibujo el cuadro y la confianza sobre el frame.
            text = f"{confidence * 100:.2f}%"
            y = startY - 10 if startY - 10 > 10 else startY + 10
            cv2.rectangle(frame_flipped, (startX, startY), (endX, endY), (0, 255, 0), 2)
            cv2.putText(frame_flipped, text, (startX, y),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 255, 0), 2)
    # Guardo el frame en el archivo de video
    out.write(frame_flipped)
    cv2.imshow("Detector de Rostros - CNN", frame_flipped)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Guardo y cierro todo
out.release()
cap.release()
cv2.destroyAllWindows()

Detector de rostros (CNN) iniciado. Presione 'q' para salir.


# Deteccion de manos con YOLO

In [12]:
cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: No se pudo abrir la cámara.")
    exit()

# modelo YOLOv8 pre-entrenado para la deteccion de manos.
model = YOLO('hand_yolov8n.pt')

# clases que el modelo puede detectar
classNames = ['Hand']

print("Detector de manos (YOLOv8) iniciado. Presione 'q' para salir.")

while True:
    # Leo un fotograma de la camara.
    ret, frame = cap.read()
    if not ret:
        break

    # Volteo el frame para un efecto espejo.
    frame = cv2.flip(frame, 1)

    # El modelo me devolvera una lista de resultados.
    results = model(frame, stream=True)

    # Itero sobre cada resultado.
    for r in results:
        # Cada resultado contiene una lista de cuadros delimitadores ('boxes').
        boxes = r.boxes
        for box in boxes:
            # --- Dibujar el cuadro delimitador ---
            x1, y1, x2, y2 = box.xyxy[0]
            x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2) # Las convierto a enteros.

            # Dibujo el rectangulo en el fotograma.
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 255), 3)

            # Obtengo la confianza de la deteccion.
            confidence = math.ceil((box.conf[0] * 100)) / 100
            # Obtengo el ID de la clase.
            cls = int(box.cls[0])
            
            # Preparo el texto de la etiqueta.
            label = f'{classNames[cls]} {confidence:.2f}'
            
            # Calculo el tamaño del texto para dibujar un fondo.
            (text_width, text_height), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.9, 2)
            
            # Dibujo un rectangulo de fondo para el texto.
            cv2.rectangle(frame, (x1, y1 - text_height - 10), (x1 + text_width, y1), (255, 0, 255), -1)
            # Pongo el texto de la etiqueta sobre el fondo.
            cv2.putText(frame, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (255, 255, 255), 2)

    # Muestro el fotograma final con las detecciones.
    cv2.imshow('Detector de Manos - YOLOv8', frame)

    # Condicion para salir del bucle.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Libero los recursos al final.
cap.release()
cv2.destroyAllWindows()

Detector de manos (YOLOv8) iniciado. Presione 'q' para salir.

0: 480x640 (no detections), 57.3ms
Speed: 13.2ms preprocess, 57.3ms inference, 0.9ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 40.5ms
Speed: 1.8ms preprocess, 40.5ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 35.7ms
Speed: 1.4ms preprocess, 35.7ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 37.3ms
Speed: 1.0ms preprocess, 37.3ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 33.2ms
Speed: 1.4ms preprocess, 33.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 36.4ms
Speed: 1.1ms preprocess, 36.4ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 31.2ms
Speed: 1.3ms preprocess, 31.2ms inference, 0.6ms postprocess per image at shape (1, 3, 480, 640)

0: 48

# Deteccion de poses

In [14]:
model = YOLO("yolov8n-pose.pt")
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    results = model(frame, verbose=False)
    annotated = results[0].plot()

    cv2.imshow("YOLOv8 Pose", annotated)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()